## pandas量化实习常用函数介绍

## 1.多线程加速问题

In [3]:
import pandas as pd
import tushare as ts

In [24]:
token_key='dc0da43c9ec5549606a55f236aa4c998c9b6f3ba9f25422f8c7b3df7'
pro=ts.pro_api(token_key)
df = pro.daily(ts_code='600000.SH', start_date='20180101', end_date='20190101')
df.head()

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
0,600000.SH,20181228,9.72,9.95,9.71,9.80,9.66,0.14,1.4493,274040.28,269118.592
1,600000.SH,20181227,9.77,9.80,9.66,9.66,9.67,-0.01,-0.1034,273475.64,266280.883
2,600000.SH,20181226,9.81,9.88,9.64,9.67,9.79,-0.12,-1.2257,205692.02,200359.184
3,600000.SH,20181225,9.77,9.86,9.60,9.79,9.91,-0.12,-1.2109,348103.97,337985.911
4,600000.SH,20181224,9.98,10.04,9.86,9.91,10.07,-0.16,-1.5889,227798.82,225706.451


In [4]:
df.empty # 查看df是否为空值

False

In [5]:
df.index=pd.to_datetime(df.trade_date)
df.sort_index(inplace=True)
df.head()

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
trade_date,,,,,,,,,,,
2018-01-02,600000.SH,20180102,12.61,12.77,12.60,12.72,12.59,0.13,1.03,313230.53,398614.966
2018-01-03,600000.SH,20180103,12.73,12.80,12.66,12.66,12.72,-0.06,-0.47,378391.01,480954.809
2018-01-04,600000.SH,20180104,12.70,12.73,12.62,12.66,12.66,0.00,0.00,278838.04,353205.838
2018-01-05,600000.SH,20180105,12.67,12.71,12.62,12.69,12.66,0.03,0.24,310267.44,393058.250
2018-01-08,600000.SH,20180108,12.69,12.71,12.63,12.68,12.69,-0.01,-0.08,313898.87,397842.209


In [23]:
for i,row in df.head().iterrows():
    print(i,'\n',row)

2018-01-02 00:00:00 
 ts_code        600000.SH
trade_date      20180102
open               12.61
high               12.77
low                 12.6
close              12.72
pre_close          12.59
change              0.13
pct_chg             1.03
vol            313230.53
amount        398614.966
square          161.7984
Name: 2018-01-02 00:00:00, dtype: object
2018-01-03 00:00:00 
 ts_code        600000.SH
trade_date      20180103
open               12.73
high                12.8
low                12.66
close              12.66
pre_close          12.72
change             -0.06
pct_chg            -0.47
vol            378391.01
amount        480954.809
square          160.2756
Name: 2018-01-03 00:00:00, dtype: object
2018-01-04 00:00:00 
 ts_code        600000.SH
trade_date      20180104
open                12.7
high               12.73
low                12.62
close              12.66
pre_close          12.66
change               0.0
pct_chg              0.0
vol            278838.04
am

In [25]:
import time

def square(row):
    close=row['close']
    time.sleep(0.01)
    return close*close

df1=df
df1['square']=df1.apply(square,axis=1)
df1.head()

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount,square
0,600000.SH,20181228,9.72,9.95,9.71,9.80,9.66,0.14,1.4493,274040.28,269118.592,96.0400
1,600000.SH,20181227,9.77,9.80,9.66,9.66,9.67,-0.01,-0.1034,273475.64,266280.883,93.3156
2,600000.SH,20181226,9.81,9.88,9.64,9.67,9.79,-0.12,-1.2257,205692.02,200359.184,93.5089
3,600000.SH,20181225,9.77,9.86,9.60,9.79,9.91,-0.12,-1.2109,348103.97,337985.911,95.8441
4,600000.SH,20181224,9.98,10.04,9.86,9.91,10.07,-0.16,-1.5889,227798.82,225706.451,98.2081


In [26]:
import threading
threads=[]
for _,row in df1.iterrows():
    t=threading.Thread(target=square,args=(row,))
    t.start()
    threads.append(t)
for t in threads:
    t.join()

In [27]:
import threading
from queue import Queue

q=Queue()

def square(row,q):
    close=row['close']
    q.put(close*close)
    time.sleep(0.1)
    
def threads_square(df,q):
    threads=[]
    for _,row in df.iterrows():
        t=threading.Thread(target=square,args=(row,q))
        t.start()
        threads.append(t)
    for t in threads:
        t.join()
    results = []
    while not q.empty():
        results.append(q.get())
    return results

df2=df
df2['square']=threads_square(df2,q)
print(df2.head())    

     ts_code trade_date  open   high   low  close  pre_close  change  pct_chg  \
0  600000.SH   20181228  9.72   9.95  9.71   9.80       9.66    0.14   1.4493   
1  600000.SH   20181227  9.77   9.80  9.66   9.66       9.67   -0.01  -0.1034   
2  600000.SH   20181226  9.81   9.88  9.64   9.67       9.79   -0.12  -1.2257   
3  600000.SH   20181225  9.77   9.86  9.60   9.79       9.91   -0.12  -1.2109   
4  600000.SH   20181224  9.98  10.04  9.86   9.91      10.07   -0.16  -1.5889   

         vol      amount   square  
0  274040.28  269118.592  96.0400  
1  273475.64  266280.883  93.3156  
2  205692.02  200359.184  93.5089  
3  348103.97  337985.911  95.8441  
4  227798.82  225706.451  98.2081  


In [28]:
df2.sample(n=20)  

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount,square
34,600000.SH,20181112,10.77,10.95,10.75,10.80,10.75,0.05,0.4651,270158.27,292478.247,116.6400
36,600000.SH,20181108,11.05,11.10,10.90,11.06,10.90,0.16,1.4679,199916.83,220506.116,122.3236
38,600000.SH,20181106,11.13,11.15,11.00,11.03,11.14,-0.11,-0.9874,184224.77,203565.760,121.6609
109,600000.SH,20180720,9.56,9.95,9.47,9.87,9.56,0.31,3.2427,332756.50,325018.010,97.4169
98,600000.SH,20180806,9.97,10.03,9.87,9.91,9.83,0.08,0.8138,174444.92,173559.265,98.2081
204,600000.SH,20180302,12.40,12.46,12.36,12.41,12.47,-0.06,-0.4800,230241.32,285473.130,154.0081
20,600000.SH,20181130,10.65,10.75,10.62,10.71,10.63,0.08,0.7526,285726.51,305468.672,114.7041
54,600000.SH,20181015,10.05,10.16,9.97,10.03,10.03,0.00,0.0000,139578.44,140122.866,100.6009
162,600000.SH,20180507,10.82,10.89,10.76,10.84,10.83,0.01,0.0900,274253.78,297162.770,117.5056
222,600000.SH,20180130,13.45,13.49,13.09,13.11,13.46,-0.35,-2.6000,1078040.18,1429924.976,171.8721


#### 注意：在使用多进程时，queue.Queue() 的 put() 和 get() 操作本身是线程安全的，但是在多进程环境中，由于多个进程之间的并发执行，无法保证 put() 和 get() 操作的顺序。因此，这两个操作可能会出现乱序的情况。

In [31]:
import threading
from queue import Queue

q=Queue()

def square(row,q):
    row_list=row.to_list()
    row_list.insert(-1,row.close**2)
    q.put(row_list)
    time.sleep(0.1)
    
def threads_square(df,q):
    threads=[]
    for _,row in df.iterrows():
        t=threading.Thread(target=square,args=(row,q))
        t.start()
        threads.append(t)
    for t in threads:
        t.join()
    results = []
    while not q.empty():
        results.append(q.get())
    return results

column=df.columns.to_list()
column.insert(-1,'square')
data=threads_square(df,q)
df_new=pd.DataFrame(data,columns=column)
print(df_new.sample(n=20))    

       ts_code trade_date   open   high    low  close  pre_close  change  \
227  600000.SH   20180123  12.75  12.92  12.68  12.90      12.77    0.13   
59   600000.SH   20181008  10.39  10.48  10.15  10.16      10.62   -0.46   
109  600000.SH   20180720   9.56   9.95   9.47   9.87       9.56    0.31   
240  600000.SH   20180104  12.70  12.73  12.62  12.66      12.66    0.00   
19   600000.SH   20181203  10.99  11.05  10.84  11.02      10.71    0.31   
41   600000.SH   20181101  11.05  11.09  10.86  10.89      10.98   -0.09   
226  600000.SH   20180124  12.97  14.00  12.89  13.57      12.90    0.67   
189  600000.SH   20180323  12.00  12.06  11.64  11.71      12.25   -0.54   
239  600000.SH   20180105  12.67  12.71  12.62  12.69      12.66    0.03   
72   600000.SH   20180911  10.22  10.27   9.98  10.02      10.18   -0.16   
184  600000.SH   20180330  11.63  11.69  11.61  11.65      11.62    0.03   
29   600000.SH   20181119  10.86  10.90  10.77  10.88      10.76    0.12   
168  600000.

[**pandas练习题网址**](https://github.com/guipsamora/pandas_exercises)

## 2.金融时间序列的问题

In [20]:
# 时间序列的处理
date = pd.date_range('20180101', '20180430')
date = pd.to_datetime(date)
datetime = date[0]

In [6]:
# 时间向后推移
dt_delta=datetime+pd.Timedelta(days=1,hours=1,minutes=1,seconds=1)
dt_replace=datetime.replace(year=2019,month=1,day=1,hour=1,minute=1,second=1)
print(datetime)
print(dt_delta)
print(dt_replace)

2018-01-01 00:00:00
2018-01-02 01:01:01
2019-01-01 01:01:01


In [10]:
# 时间向后延迟
print(datetime+pd.DateOffset(days=10))
# 时间的替换
print(datetime+pd.DateOffset(day=5))

2018-01-11 00:00:00
2018-01-05 00:00:00


In [12]:
datetime.strftime('%Y%m%d')
datetime.strftime('%Y-%m-%d')

'2018-01-01'

In [18]:
datetime.date()
datetime.time()

datetime.time(0, 0)

In [21]:
datetime.today()

Timestamp('2024-03-15 12:58:20.729389')

## 3.占位符问题

In [23]:
# 小数占位符
"{:.2f}".format(1.23456789)

'1.23'

In [24]:
# 字符串占位符
"%s"%1.23

'1.23'

In [25]:
# 整数占位符
"%d"%1.23

'1'

In [26]:
"{0},{1},{0}".format(1,2)

'1,2,1'

In [28]:
code=123
name='小花'
f'{code},{name}'

'123,小花'